# Sensitibity Analysis

We are going to use only three variables $I,\, P$ and $E$, each of them is a valid boolean string such that together they form a valid scenario (taking into account no more than 2 ones in each variable).

In [1]:
import numpy as np
import itertools
import pandas as pd
import warnings
import random

import datetime as dt
import time

import os
from os import listdir
from os.path import isfile,join
import matplotlib.pyplot as plt

In [2]:
PATH = '/home/moni/Documents/motmo/timeSeries_files/' # original data
PATH2 = '/home/moni/Documents/motmo/data_without_hhID/' # folder in which we will store transformed data

### Creating input space
- $I$ and $P$ are boolean arrays of size three so that they have no 2 consecutive ones.
- $E$ is a boolean array of size 4. 

In total, there are $7*7*11=539$ possible scenarios.

In [3]:
s3 = list(itertools.product(range(2), repeat=3))
s4 = list(itertools.product(range(2), repeat=4))
l1 = [i for i in s3 if sum(i)<3]
l2 = l1
l3 = [i for i in s4 if sum(i)<3]
L=[]
L.append(l1)
L.append(l2)
L.append(l3)
inputs = list(itertools.product(*L)) # here we have the 539 scenarios

In [41]:
s3

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [24]:
# step = 1/6
# I = list(range(0,7,1))
# I = [i/6 for i in I]
P = I
E = list(range(0,11,1))
E = [i/10 for i in E]
E

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [47]:
dict_I = dict(zip(l1,I))
dict_P = dict_I
dict_E = dict(zip(l3,E))
dict_E

{(0, 0, 0, 0): 0.0,
 (0, 0, 0, 1): 0.1,
 (0, 0, 1, 0): 0.2,
 (0, 0, 1, 1): 0.3,
 (0, 1, 0, 0): 0.4,
 (0, 1, 0, 1): 0.5,
 (0, 1, 1, 0): 0.6,
 (1, 0, 0, 0): 0.7,
 (1, 0, 0, 1): 0.8,
 (1, 0, 1, 0): 0.9,
 (1, 1, 0, 0): 1.0}

In [57]:
# d_inputs = [(dict_I[i], dict_P[i], dict_E[k]) for i in
inputs_d = []
k=0
for i in inputs:
    t = [dict_I[i[0]], dict_P[i[1]], dict_E[i[2]]]
    inputs_d.append(t)
inputs_d

[[0.0, 0.0, 0.0],
 [0.0, 0.0, 0.1],
 [0.0, 0.0, 0.2],
 [0.0, 0.0, 0.3],
 [0.0, 0.0, 0.4],
 [0.0, 0.0, 0.5],
 [0.0, 0.0, 0.6],
 [0.0, 0.0, 0.7],
 [0.0, 0.0, 0.8],
 [0.0, 0.0, 0.9],
 [0.0, 0.0, 1.0],
 [0.0, 0.16666666666666666, 0.0],
 [0.0, 0.16666666666666666, 0.1],
 [0.0, 0.16666666666666666, 0.2],
 [0.0, 0.16666666666666666, 0.3],
 [0.0, 0.16666666666666666, 0.4],
 [0.0, 0.16666666666666666, 0.5],
 [0.0, 0.16666666666666666, 0.6],
 [0.0, 0.16666666666666666, 0.7],
 [0.0, 0.16666666666666666, 0.8],
 [0.0, 0.16666666666666666, 0.9],
 [0.0, 0.16666666666666666, 1.0],
 [0.0, 0.3333333333333333, 0.0],
 [0.0, 0.3333333333333333, 0.1],
 [0.0, 0.3333333333333333, 0.2],
 [0.0, 0.3333333333333333, 0.3],
 [0.0, 0.3333333333333333, 0.4],
 [0.0, 0.3333333333333333, 0.5],
 [0.0, 0.3333333333333333, 0.6],
 [0.0, 0.3333333333333333, 0.7],
 [0.0, 0.3333333333333333, 0.8],
 [0.0, 0.3333333333333333, 0.9],
 [0.0, 0.3333333333333333, 1.0],
 [0.0, 0.5, 0.0],
 [0.0, 0.5, 0.1],
 [0.0, 0.5, 0.2],
 [0.0, 0.5,

### Sampling
### Variance decomposition
We see this model as a function $f(X)=Y$, where the inputs are the scenarios, so $X\in\mathbb{R}^d$ and the output is a real value, that in our case, can be the emissions or the mobility choices. First, we are going to do an analysis on (overall) emissions and when we implement it, we can use the othe outputs.

The idea consists in estimating certain variances that we are going to define later in order to compute some sesitivity indices (there are first and second order sensitivity indices). We write
$$Var(Y)=\sum_{i=1}^dV_i+\sum_{i<j}^dV_{ij}+\ldots+V_{1,2, \ldots,d}$$where
$$V_i=Var_{x_i}(E_{X_{\sim i}}(Y|X))$$where the $X_{\sim i}$ notation means the set of all variables except $x_i$.

#### First Order Index
$$S_i= \frac{V_i}{Var(Y)}$$We can interpret it as follows: "the fractional reduction in the variance of $Y$ which would be obtained on average if $X$ could be fixed".

### Estimating
The Monte Carlo method offers an estimation based on sampling. The first thing is to take $N$ pairs of samples (inputs) at random. However, in our case, there is a problem with this approximation, because later on we will create another $d$ matrices that will consist on replacing one column of $A$ with a column of $B$ given some indices. The thing is that it is very likely that this sampling will generate scenarios that are not possible due to the fact that we cannot choose in the same category 3 options turned to 1. Therefore, my idea is to first sample uniformly at random $A$, and then sample $B$ based on some probabilities that I will create accordingly.
1. First we should sample $N$ inputs (scenarios) and then create two $N\times d$ matrices $A$ and $B$ (be careful with the sampling $B$).
2. Build $d$ further $N\times d$ matrices $A_B^i$, for $i = 1,2,\ldots,d$, such that the $i$th column of $A_B^i$ is equal to the $i$th column of $B$, and the remaining columns are from $A$.
3. The $A, \,B$, and the $d$ $A_B^i$ matrices in total specify $N\cdot(d+2)$ points in the input space (one for each row).  Then we compute the corresponding $f(A)$, $f(B)$ and $f(A_B^i)$ values (that is, compute the emissions of each scenario).

To compute the variance, we can use the following estimator: $$Var_{x_i}(E_{X_{\sim i}}(Y|X))\approx \frac{1}{N}\sum_{i=1}^Nf(B)_j\left(f(A_B^i)_j-f(A)_j\right)$$

In [69]:
def sampling(in_space,N):
    AB = random.sample(in_space,2*N)
    A = AB[0:N]
    B = AB[N:2*N]
    return A,B

In [78]:
# ABtest = random.sample(inputs_d,2*8)
# Atest = ABtest[0:8]
# Btest = ABtest[8:16]
# Atest
# ABtest = np.zeros((3,8,3))
# Ant,Bnt = np.array(Atest), np.array(Btest)
# ABtest[0] = Ant
# ABtest[0][:,0] = Bnt[:,0]
ABtest[1]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [77]:
Ant

array([[0.        , 0.33333333, 0.9       ],
       [0.5       , 0.33333333, 0.6       ],
       [0.66666667, 0.33333333, 0.2       ],
       [0.5       , 0.        , 0.1       ],
       [0.66666667, 1.        , 0.5       ],
       [0.33333333, 0.5       , 0.9       ],
       [0.16666667, 0.16666667, 0.4       ],
       [0.16666667, 0.16666667, 0.5       ]])

In [79]:
def mat_AB_i(A,B): 
    d = 3
    N = len(A)
    An,Bn = np.array(A), np.array(B)
    AB_mat = np.zeros((d,N,d))
    for i in range(0,d):
        AB_mat[i] = An
        AB_mat[i][:,i] = Bn[:,i]
    return AB_mat

In [80]:
mat_AB_i(Atest,Btest)

array([[[0.16666667, 0.33333333, 0.9       ],
        [0.66666667, 0.33333333, 0.6       ],
        [0.33333333, 0.33333333, 0.2       ],
        [0.83333333, 0.        , 0.1       ],
        [0.83333333, 1.        , 0.5       ],
        [0.66666667, 0.5       , 0.9       ],
        [0.83333333, 0.16666667, 0.4       ],
        [0.        , 0.16666667, 0.5       ]],

       [[0.        , 0.5       , 0.9       ],
        [0.5       , 0.5       , 0.6       ],
        [0.66666667, 0.66666667, 0.2       ],
        [0.5       , 0.33333333, 0.1       ],
        [0.66666667, 1.        , 0.5       ],
        [0.33333333, 0.16666667, 0.9       ],
        [0.16666667, 0.5       , 0.4       ],
        [0.16666667, 0.66666667, 0.5       ]],

       [[0.        , 0.33333333, 1.        ],
        [0.5       , 0.33333333, 0.        ],
        [0.66666667, 0.33333333, 0.6       ],
        [0.5       , 0.        , 0.8       ],
        [0.66666667, 1.        , 1.        ],
        [0.33333333, 0.5      

In [82]:
Atest

[[0.0, 0.3333333333333333, 0.9],
 [0.5, 0.3333333333333333, 0.6],
 [0.6666666666666666, 0.3333333333333333, 0.2],
 [0.5, 0.0, 0.1],
 [0.6666666666666666, 1.0, 0.5],
 [0.3333333333333333, 0.5, 0.9],
 [0.16666666666666666, 0.16666666666666666, 0.4],
 [0.16666666666666666, 0.16666666666666666, 0.5]]